# Stock Screener

### This notebook is intended to help get a centralized view of important statistics for stocks of companies.

### 1. Imports
We begin with all the imports we will need for analysis.

In [55]:
import pickle
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

### 2. Getting tickers for gold stocks.

In [77]:
#Where tickers comes from
goldStocksUrl = "http://www.miningfeeds.com/gold-mining-report-all-countries"

def goldStockParser(url, tickerColumn, stockTypes):
    """Goes to url above and pulls the tickers of each 
       gold mining company, dumps them in a pickle file 
       and returns the tickers as a list."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find("table")
    tickers = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[tickerColumn].text
        #Randgold real ticker.
        if (ticker == "GOLD.L"):
            ticker = "GOLD"
        tickers.append(ticker)

    with open(stockTypes + ".pickle", "wb") as file:
        pickle.dump(tickers, file)

    return tickers

def pickleLoader(stockTypes):
    """Loads tickers from a pickle file and returns them as a list."""
    with open(stockTypes + ".pickle", "rb") as file:
        return pickle.load(file)

#Look in cache if False, go to website if true.    
reloadGoldTickers = False

badDataStocks = ["GLF.AX", "AZ.TO", "LEX.TO"]
goldStockTickers = []

if reloadGoldTickers:
    goldStockTickers = goldStockParser(goldStocksUrl, 2, "goldStocks")
else:
    goldStockTickers = pickleLoader("goldStocks")

#Remove stocks with bad data.    
for ticker in badDataStocks:
    goldStockTickers.remove(ticker)
    
print(goldStockTickers)

['NEM', 'ABX', 'FNV.TO', 'NCM.AX', 'GG', 'AEM', 'KGC', 'RGLD', 'RRS.L', 'FRES.L', 'AGG.AX', 'EVN.AX', 'GFI', 'BTO.TO', 'YRI.TO', 'IMG.TO', 'AGI.TO', 'CEY.L', 'PVG', 'DGC.TO', 'NGD.TO', 'RRL.AX', 'OGC.TO', 'SBM.AX', 'CG.TO', 'EDV.TO', 'NG.TO', 'ELD.TO', 'SAR.AX', 'SMF.TO', 'HMY', 'TXG.TO', 'CGG.TO', 'NSU.TO', 'GUY.TO', 'RSG.AX', 'SEA.TO', 'MUX', 'RIC', 'PG.TO', 'OSK.TO', 'ALIAF', 'CNL.TO', 'GOR.AX', 'POG.L', 'DNA.TO', 'GSS', 'PAF.L', 'AR.TO', 'PRU.AX', 'TBR.AX', 'BGM.V', 'TGZ.TO', 'SBB.TO', 'PRH.AX', 'GORO', 'WDO.TO', 'HRT.TO', 'WAF.AX', 'VIT.V', 'AOT.V', 'NGQ.TO', 'BDR.AX', 'HUM.L', 'RMS.AX', 'SLR.AX', 'BSX.TO', 'GCY.AX', 'KOR.TO', 'MOZ.TO', 'GBU.TO', 'BTR.V', 'HRR.AX', 'PRB.V', 'ALK.AX', 'HAS.AX', 'R.TO', 'RPM.V', 'ARZ.TO', 'JAG.TO', 'MAX.TO', 'TNG.AX', 'PEN.AX', 'CAL.TO', 'TLG.AX', 'RUP.V', 'GGP.L', 'AAL.V', 'KCN.AX', 'ITH.TO', 'MML.AX', 'GGG.AX', 'VGZ', 'ARU.V', 'RMX.TO', 'QBL.AX', 'TSG.L', 'ATC.V', 'KGL.AX', 'TML.TO', 'IMA.AX', 'DRM.AX', 'RED.AX', 'MTO.V', 'DGR.AX', 'MRC.AX', 'DNG.

### 3. Specify formatting details of data.

Specify rows of data that are wanted from yahoo.

Specify rows of data that have letters in their numbers for conversion.

Specify rows of data that have percentage signs in their numbers for conversion.

In [78]:
wantedRows = {0  : "marketCap", 
              1  : "enterpriseValue",
              2  : "trailingPe",
              3  : "forwardPe",
              4  : "5YrPeg",
              5  : "priceSales",
              6  : "priceBook",
              7  : "evSales",
              8  : "evEBITDA",
              11 : "profitMargin",
              12 : "operatingMargin",
              13 : "returnOnAssets",
              14 : "returnOnEquity",
              15 : "revenue",
              16 : "revenuePerShare",
              17 : "quarterlyRevenueGrowth",
              18 : "grossProfit",
              19 : "EBITDA",
              20 : "netIncomeForCommon",
              21 : "Diluted EPS",
              22 : "quarterlyEarningsGrowth",
              23 : "totalCash",
              24 : "cashPerShare",
              25 : "totalDebt",
              26 : "debtEquity",
              27 : "currentRatio",
              28 : "bookValuePerShare", 
              29 : "operatingCashFlow",
              30 : "leveredFreeCashFlow",
              31 : "beta3YrMonthly",
              32 : "52WeekChange",
              33 : "spy52WeekChange",
              34 : "52WeekHigh",
              35 : "52WeekLow",
              36 : "50dayMvAvg",
              37 : "200dayMvAvg",
              38 : "avg3MonthVol",
              39 : "avg10DayVol",
              40 : "sharesOutstanding",
              41 : "sharesFloating",
              42 : "heldByInsiders",
              43 : "heldByInstitutions",
              44 : "sharesShort1MonthPrior",
              45 : "shortRatio",
              46 : "shortToFloat",
              47 : "shortToOutstanding",
              48 : "sharesShort2MonthsPrior",
              49 : "forwardDividendRate",
              50 : "forwardDividendYield",
              51 : "trailingDividendRate",
              52 : "trailingDividendYield",
              53 : "5yrAvgDividendYield",
              54 : "payoutRatio",
              55 : "dividendDate",
              56 : "exDividendDate",
              57 : "lastSplitFactorNewPerOld",
              58 : "lastSplitDate",
              60 : "currentPrice",}

letterNumbers = ["marketCap", 
                 "enterpriseValue",
                 "revenue",
                 "grossProfit",
                 "EBITDA",
                 "netIncomeForCommon",
                 "totalCash",
                 "totalDebt",
                 "operatingCashFlow",
                 "leveredFreeCashFlow",
                 "avg3MonthVol",
                 "avg10DayVol",
                 "sharesOutstanding",
                 "sharesFloating",
                 "sharesShort1MonthPrior",
                 "sharesShort2MonthsPrior",]

percentageNumbers = ["profitMargin",
                     "operatingMargin",
                     "returnOnAssets",
                     "returnOnEquity",
                     "quarterlyRevenueGrowth",
                     "quarterlyEarningsGrowth"
                     "52WeekChange",
                     "spy52WeekChange",
                     "heldByInsiders",
                     "heldByInstitutions",
                     "shortToFloat",
                     "shortToOutstanding",
                     "forwardDividendYield",
                     "trailingDividendYield",
                     "payoutRatio",]


We need to look up data from yahoo and store it in a csv locally for caching purposes. Also want to format well. ie remove redundant columns and remove symbols such as B and %.

In [79]:
baseurl = "https://uk.finance.yahoo.com/quote/"

def getDataFromYahoo(stockTypes, tickers, baseurl = baseurl):
    """Checks if data has been stored in csv, if not
       looks to yahoo. Then returns as map of tickers to dataframe."""
    #Create directory for data of tickers.
    dataDir = "./%s_data" % stockTypes
    createDirIfItDoesntExist(dataDir)
    
    frames = {}
    
    for ticker in tickers:
        dataFile = "./%s/%s.csv" % (dataDir, ticker)
        data = None
        if not os.path.exists(dataFile):
            exists, data = saveDataAndReturnSeries(dataFile, ticker)
            if not exists:
                continue
        else:ii
            print("Already have %s" % ticker)
            data = pd.read_csv(dataFile, index_col=0)
        frames[ticker] = data
        with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
            print(data)
    return frames

def createDirIfItDoesntExist(dataDir):
    """If directory with name dataDir does not exist, create it."""
    if not os.path.exists(dataDir):
        os.makedirs(dataDir)
        
def reformatData(data):
    """Reformat data with better names and remove redundancies"""
    dataDict = {}
    for index in wantedRows:
        series = data.loc[index]
        key = wantedRows[index]
        value = series.get(1)
        dataDict[key] = value
        dataDict["ticker"] = ticker
    return pd.Series(dataDict)

def getCurrency(html):
    soup = bs(html)
    results = soup.findAll("span", {"data-reactid" : "9"})
    i = 0
    for result in results:
        if i == 1:
            line = result.text
        i += 1
    return line[-3:] 
        
def saveDataAndReturnSeries(dataFile, ticker):
    """Gets data from yahoo and dumps in csv file before 
       returning data as pandas series. Returns true, series if
       there is data, false, none if not."""
    print("Requesting: %s" % ticker)

    #Go to summary page and scrape open.
    priceData = requests.get(baseurl + "%s?p=%s" % (ticker, ticker))       
    summaryFrames = pd.read_html(priceData.text)
    summaryFrame = pd.concat(summaryFrames, ignore_index=True)
    priceFrame = summaryFrame.loc[1:1]
    
    #Scrape currency from summary page.
    currency = getCurrency(priceData.text)
    
    #Go to stats page and scrape stats.
    data = requests.get(baseurl + "%s/key-statistics?p=%s" % (ticker, ticker))
    frames = pd.read_html(data.text)
    if len(frames) < 3:
        print("No data for: %s" % ticker)
        return False , None

    #Join stats and price
    frames.append(priceFrame)
    frame = pd.concat(frames, ignore_index=True)
    
    #Reformat
    reformattedSeries = reformatData(frame)
    reformattedSeries["Currency"] = currency
    print(reformattedSeries)
    
    #Save data
    reformattedSeries.to_csv(dataFile)
    return True, reformattedSeries

goldTickerData = getDataFromYahoo("gold_stocks", goldStockTickers)

Requesting: NEM
marketCap                        17.23B
ticker                           LEX.TO
enterpriseValue                  19.38B
trailingPe                          NaN
forwardPe                         23.78
5YrPeg                             6.64
priceSales                         2.42
priceBook                          1.63
evSales                            2.73
evEBITDA                           7.17
profitMargin                     -2.63%
operatingMargin                  16.01%
returnOnAssets                    3.44%
returnOnEquity                   -1.62%
revenue                           7.11B
revenuePerShare                   13.34
quarterlyRevenueGrowth           -8.10%
grossProfit                       3.33B
EBITDA                             2.7B
netIncomeForCommon                -250M
Diluted EPS                       -0.35
quarterlyEarningsGrowth             NaN
totalCash                         3.13B
cashPerShare                       5.87
totalDebt               

marketCap                        17.15B
ticker                           LEX.TO
enterpriseValue                  15.61B
trailingPe                        80.63
forwardPe                         71.32
5YrPeg                             8.11
priceSales                        25.59
priceBook                          3.61
evSales                           23.30
evEBITDA                          30.06
profitMargin                     31.91%
operatingMargin                  40.19%
returnOnAssets                    3.51%
returnOnEquity                    4.55%
revenue                            670M
revenuePerShare                    3.60
quarterlyRevenueGrowth           -0.60%
grossProfit                        533M
EBITDA                           519.3M
netIncomeForCommon               213.8M
Diluted EPS                        1.14
quarterlyEarningsGrowth         -13.20%
totalCash                         76.9M
cashPerShare                       0.41
totalDebt                           NaN


marketCap                         8.19B
ticker                           LEX.TO
enterpriseValue                  11.01B
trailingPe                       104.27
forwardPe                         22.10
5YrPeg                            25.83
priceSales                         2.63
priceBook                          0.58
evSales                            3.54
evEBITDA                          10.31
profitMargin                      2.47%
operatingMargin                 -22.65%
returnOnAssets                   -2.02%
returnOnEquity                    0.55%
revenue                           3.11B
revenuePerShare                    3.58
quarterlyRevenueGrowth          -28.30%
grossProfit                       1.55B
EBITDA                            1.07B
netIncomeForCommon                  77M
Diluted EPS                        0.09
quarterlyEarningsGrowth             NaN
totalCash                          166M
cashPerShare                       0.19
totalDebt                         2.97B


marketCap                         3.38B
ticker                           LEX.TO
enterpriseValue                   4.42B
trailingPe                        15.66
forwardPe                         22.58
5YrPeg                            -3.55
priceSales                         1.04
priceBook                          0.74
evSales                            1.36
evEBITDA                           3.99
profitMargin                      6.85%
operatingMargin                   0.46%
returnOnAssets                    0.12%
returnOnEquity                    4.88%
revenue                           3.24B
revenuePerShare                    2.59
quarterlyRevenueGrowth           -8.90%
grossProfit                       1.55B
EBITDA                            1.11B
netIncomeForCommon               221.7M
Diluted EPS                        0.17
quarterlyEarningsGrowth             NaN
totalCash                          566M
cashPerShare                       0.45
totalDebt                         1.73B


marketCap                         5.92B
ticker                           LEX.TO
enterpriseValue                   5.71B
trailingPe                        24.29
forwardPe                       1848.97
5YrPeg                             2.69
priceSales                         4.88
priceBook                        159.69
evSales                            4.70
evEBITDA                           9.80
profitMargin                     20.30%
operatingMargin                  31.10%
returnOnAssets                    5.59%
returnOnEquity                    7.20%
revenue                           1.21B
revenuePerShare                   12.87
quarterlyRevenueGrowth           -7.10%
grossProfit                     511.92M
EBITDA                          582.74M
netIncomeForCommon              246.38M
Diluted EPS                      258.10
quarterlyEarningsGrowth          26.10%
totalCash                       653.53M
cashPerShare                       6.92
totalDebt                         2.77M


...

marketCap                      346.43M
ticker                          LEX.TO
enterpriseValue                387.92M
trailingPe                       12.29
forwardPe                        20.12
5YrPeg                            3.16
priceSales                        1.07
priceBook                         0.57
evSales                           1.19
evEBITDA                          2.92
profitMargin                     8.68%
operatingMargin                 15.50%
returnOnAssets                   3.64%
returnOnEquity                   4.68%
revenue                        324.77M
revenuePerShare                   3.02
quarterlyRevenueGrowth           5.20%
grossProfit                    122.99M
EBITDA                         132.99M
netIncomeForCommon              28.19M
Diluted EPS                       0.26
quarterlyEarningsGrowth        -24.10%
totalCash                       80.24M
cashPerShare                      0.75
totalDebt                       76.17M
debtEquity               

marketCap                      NaN
ticker                      LEX.TO
enterpriseValue                NaN
trailingPe                     NaN
forwardPe                      NaN
5YrPeg                         NaN
priceSales                     NaN
priceBook                      NaN
evSales                        NaN
evEBITDA                       NaN
profitMargin                   NaN
operatingMargin                NaN
returnOnAssets                 NaN
returnOnEquity                 NaN
revenue                        NaN
revenuePerShare                NaN
quarterlyRevenueGrowth         NaN
grossProfit                    NaN
EBITDA                         NaN
netIncomeForCommon             NaN
Diluted EPS                    NaN
quarterlyEarningsGrowth        NaN
totalCash                      NaN
cashPerShare                   NaN
totalDebt                      NaN
debtEquity                     NaN
currentRatio                   NaN
bookValuePerShare              NaN
operatingCashFlow   

marketCap                       495.88M
ticker                           LEX.TO
enterpriseValue                 460.86M
trailingPe                        42.30
forwardPe                         17.52
5YrPeg                              NaN
priceSales                         4.20
priceBook                          3.23
evSales                            3.90
evEBITDA                          10.98
profitMargin                      9.86%
operatingMargin                  21.02%
returnOnAssets                    8.31%
returnOnEquity                    7.98%
revenue                         118.12M
revenuePerShare                    0.88
quarterlyRevenueGrowth           36.60%
grossProfit                      17.95M
EBITDA                           41.98M
netIncomeForCommon               11.65M
Diluted EPS                        0.09
quarterlyEarningsGrowth       1,126.70%
totalCash                        30.71M
cashPerShare                       0.23
totalDebt                         8.47M


marketCap                      200.34M
ticker                          LEX.TO
enterpriseValue                161.48M
trailingPe                         NaN
forwardPe                       -29.00
5YrPeg                             NaN
priceSales                      320.23
priceBook                         5.27
evSales                         258.11
evEBITDA                         -7.21
profitMargin                     0.00%
operatingMargin             -3,599.38%
returnOnAssets                 -45.14%
returnOnEquity                 -94.36%
revenue                        625.62k
revenuePerShare                   0.00
quarterlyRevenueGrowth         434.20%
grossProfit                    412.78k
EBITDA                          -22.4M
netIncomeForCommon             -22.12M
Diluted EPS                      -0.04
quarterlyEarningsGrowth            NaN
totalCash                       35.41M
cashPerShare                      0.05
totalDebt                          NaN
debtEquity               

KeyboardInterrupt: 